In [1]:
# This file is the scratch space for exploring and trying various code on the data

## Overall notes
- Data size is < 5Gb for the whole file, it can fit in memory easily
- Monthly differential data available, we do not need to run the through the whole file every time
  - Merging can be beneficial because
    1. Little to no risk of going out of memory
    2. Use much less internet bandwidth to download the data
    3. Use much less computational resource by omitting processed data
- It is beneficial to have a queriable intermediate storage (e.g. SQL table) for this
  - We might add longitude and latitude for distance-based querying for the properties

In [17]:
pip install more_itertools

Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

In [18]:
from more_itertools import first

In [139]:
headers = [
    "tid",
    "price",
    "date",
    "postcode",
    "property_type",
    "is_new",
    "freehold/leasehold",
    "paon",
    "saon",
    "street",
    "locality",
    "town/city",
    "district",
    "county",
    "ppd_category_type",
    "record_status",
]

In [ ]:
df = first(pd.read_csv("data/pp-complete.csv", chunksize=100000, names=headers))

In [39]:
# Find some popular postcodes to test with address id and duplication on address, use set for performance
most_popular_postcodes = set(
    df.groupby("postcode").count().sort_values("tid", ascending=False).head(10).index
)
most_popular_postcodes

{'CM21 9PF',
 'E3 2UR',
 'IP2 8RS',
 'NE32 4SW',
 'OX26 4US',
 'SE1 3HS',
 'SG1 2AJ',
 'SG1 2AL',
 'SW1V 2DN',
 'TW3 4BZ'}

In [42]:
df = pd.DataFrame()

# get all the transaction in said postcode for testing purpose
for subdf in pd.read_csv("data/pp-complete.csv", chunksize=100000, names=headers):
    df = pd.concat([df, subdf[subdf["postcode"].isin(most_popular_postcodes)]])

In [45]:
# see if paon and postcode is enough as a unique id
df.groupby(["paon", "postcode"]).nunique().sort_values("tid", ascending=False)

,,tid,price,date,property_type,is_new,freehold/leasehold,saon,street,locality,town/city,district,county,ppd_category_type,record_status
paon,postcode,,,,,,,,,,,,,,
PARK WEST BUILDING,E3 2UR,185,135,150,1,2,1,59,1,1,1,1,1,1,1
"WEBSTER HOUSE, 26",SW1V 2DN,37,32,24,1,2,1,12,1,1,1,1,1,1,1
"BONINGTON HOUSE, 16",SW1V 2DN,33,28,30,1,2,1,12,1,1,1,1,1,2,1
MARYGOLD HOUSE,TW3 4BZ,33,27,31,1,2,1,12,1,1,1,1,1,1,1
"FARINGTON HOUSE, 22",SW1V 2DN,33,31,30,1,2,1,12,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,SE1 3HS,1,1,1,1,1,1,0,1,1,1,1,1,1,1
39,NE32 4SW,1,1,1,1,1,1,0,1,1,1,1,1,1,1
4,IP2 8RS,1,1,1,1,1,1,0,1,1,1,1,1,1,1


In [55]:
# understand why and how the saon differs with the same paon
index = (
    df.groupby(["paon", "postcode"])
    .nunique()
    .sort_values("tid", ascending=False)
    .query("saon >= 1")
    .index
)
df.groupby(["paon", "postcode"])["saon"].apply(set)[index]

paon                 postcode
PARK WEST BUILDING   E3 2UR      {FLAT 112, FLAT 16, FLAT 37, FLAT 21, FLAT 310...
WEBSTER HOUSE, 26    SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 6, FLAT 12, FLAT...
BONINGTON HOUSE, 16  SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 7, FLAT 12, FLAT...
MARYGOLD HOUSE       TW3 4BZ     {FLAT 8, FLAT 9, FLAT 1, FLAT 12, FLAT 7, FLAT...
FARINGTON HOUSE, 22  SW1V 2DN    {FLAT 8, FLAT 9, FLAT 6, FLAT 1, FLAT 12, FLAT...
RAEBURN HOUSE, 20    SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 7, FLAT 12, FLAT...
HARDING HOUSE, 24    SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 7, FLAT 12, FLAT...
HERRING HOUSE, 18    SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 6, FLAT 12, FLAT...
CONSTABLE HOUSE, 14  SW1V 2DN    {FLAT 8, FLAT 9, FLAT 1, FLAT 12, FLAT 7, FLAT...
FOXTAIL HOUSE        TW3 4BZ     {FLAT 8, FLAT 9, FLAT 1, FLAT 12, FLAT 7, FLAT...
CLOVER HOUSE         TW3 4BZ     {FLAT 8, FLAT 1, FLAT 6, FLAT 2, 4, FLAT 3, FL...
BOW QUARTER, 60      E3 2UR      {FLAT 51 PARK WEST BUILD

In [60]:
# Check if postcode, paon and saon are unique
df.groupby(["postcode", "paon", "saon"]).nunique().sort_values("tid", ascending=False)

# Found edge case: PARK WEST BUILDING can be mentioned on either paon or saon. There might be many more cases of such
# These requires more effort than this exercise so is not handled here
# Initial suggested solution is to find out common way to describe flat number (e.g.: /FLAT\s?\d+/), normalise it (only use \d+), and get the remaining sting for further deduplication
# To be confirmed:
# - Does the remainder string matter?
# - Are there multiple way to refer to the same flat number? (e.g.: 43 vs forty three)
# - What about using alphabets? (e.g.: FLAT A, FLAT 1A)

tid  price  date  \
postcode paon               saon                                           
E3 2UR   PARK WEST BUILDING FLAT 43                       6      6     6   
                            FLAT 8                        6      6     6   
                            FLAT 2                        6      6     6   
                            FLAT 17                       6      6     6   
                            FLAT 1                        6      6     6   
...                                                     ...    ...   ...   
TW3 4BZ  FOXTAIL HOUSE      FLAT 1                        1      1     1   
SW1V 2DN RAEBURN HOUSE, 20  FLAT 4                        1      1     1   
E3 2UR   PARK WEST BUILDING FLAT 212                      1      1     1   
SW1V 2DN HERRING HOUSE, 18  FLAT 11                       1      1     1   
E3 2UR   BOW QUARTER, 60    FLAT 11 PARK WEST BUILDING    1      1     1   

                                                        property_type  is_new  \
postcode paon               saon                                                
E3 2UR   PARK WEST BUILDING FLAT 43                                 1       2   
                            FLAT 8                                  1       2   
                            FLAT 2                                  1       2   
                            FLAT 17                                 1       2   
                            FLAT 1                                  1       2   
...                                                               ...     ...   
TW3 4BZ  FOXTAIL HOUSE      FLAT 1                                  1       1   
SW1V 2DN RAEBURN HOUSE, 20  FLAT 4                                  1       1   
E3 2UR   PARK WEST BUILDING FLAT 212                                1       1   
SW1V 2DN HERRING HOUSE, 18  FLAT 11                                 1       1   
E3 2UR   BOW QUARTER, 60    FLAT 11 PARK WEST BUILDING              1       1   

                                                        freehold/leasehold  \
postcode paon               saon                                             
E3 2UR   PARK WEST BUILDING FLAT 43                                      1   
                            FLAT 8                                       1   
                            FLAT 2                                       1   
                            FLAT 17                                      1   
                            FLAT 1                                       1   
...                                                                    ...   
TW3 4BZ  FOXTAIL HOUSE      FLAT 1                                       1   
SW1V 2DN RAEBURN HOUSE, 20  FLAT 4                                       1   
E3 2UR   PARK WEST BUILDING FLAT 212                                     1   
SW1V 2DN HERRING HOUSE, 18  FLAT 11                                      1   
E3 2UR   BOW QUARTER, 60    FLAT 11 PARK WEST BUILDING                   1   

                                                        street  locality  \
postcode paon               saon                                           
E3 2UR   PARK WEST BUILDING FLAT 43                          1         1   
                            FLAT 8                           1         1   
                            FLAT 2                           1         1   
                            FLAT 17                          1         1   
                            FLAT 1                           1         1   
...                                                        ...       ...   
TW3 4BZ  FOXTAIL HOUSE      FLAT 1                           1         1   
SW1V 2DN RAEBURN HOUSE, 20  FLAT 4                           1         1   
E3 2UR   PARK WEST BUILDING FLAT 212                         1         1   
SW1V 2DN HERRING HOUSE, 18  FLAT 11                          1         1   
E3 2UR   BOW QUARTER, 60    FLAT 11 PARK WEST BUILDING       1         0   

         

In [107]:
from typing import Hashable
import math


# use a function to create the pid as this might change for the lack of data consistency
def create_pid(row) -> Hashable:
    match row["saon"]:
        case x if isinstance(x, float) and math.isnan(x):
            return f'{row["postcode"]}, {row["paon"]}'
        case x:
            return f'{row["postcode"]}, {row["paon"]}, {x}'

In [108]:
df["pid"] = df.apply(create_pid, axis=1)

In [111]:
# find example where multiple transactions happens to the same property
df.groupby("pid").count().sort_values("tid", ascending=False).head(20)

,tid,price,date,postcode,property_type,is_new,freehold/leasehold,paon,saon,street,locality,town/city,district,county,ppd_category_type,record_status
pid,,,,,,,,,,,,,,,,
"CM21 9PF, 130",8,8,8,8,8,8,8,8,0,8,4,8,8,8,8,8
"SG1 2AL, 102",8,8,8,8,8,8,8,8,0,8,5,8,8,8,8,8
"SG1 2AJ, 15",7,7,7,7,7,7,7,7,0,7,5,7,7,7,7,7
"OX26 4US, 123",7,7,7,7,7,7,7,7,0,7,4,7,7,7,7,7
"CM21 9PF, 135",7,7,7,7,7,7,7,7,0,7,6,7,7,7,7,7
"NE32 4SW, 56",6,6,6,6,6,6,6,6,0,6,3,6,6,6,6,6
"SG1 2AJ, 53",6,6,6,6,6,6,6,6,0,6,4,6,6,6,6,6
"OX26 4US, 46",6,6,6,6,6,6,6,6,0,6,4,6,6,6,6,6
"SG1 2AL, 103",6,6,6,6,6,6,6,6,0,6,3,6,6,6,6,6


In [142]:
for one in df.groupby("pid"):
    break

In [148]:
row

Pandas(Index=2376593, tid='{9610C0A4-7650-479B-AEA8-3B953CF7AD71}', price=51500, date='1997-06-20 00:00', postcode='CM21 9PF', property_type='F', is_new='N', _7='L', paon='100', saon=nan, street='LAWRENCE MOORINGS', locality='SAWBRIDGEWORTH', _12='SAWBRIDGEWORTH', district='EPPING FOREST', county='ESSEX', ppd_category_type='A', record_status='A', pid='CM21 9PF, 100')

In [144]:
for row in one[1].itertuples():
    print(row)

Pandas(Index=2376593, tid='{9610C0A4-7650-479B-AEA8-3B953CF7AD71}', price=51500, date='1997-06-20 00:00', postcode='CM21 9PF', property_type='F', is_new='N', _7='L', paon='100', saon=nan, street='LAWRENCE MOORINGS', locality='SAWBRIDGEWORTH', _12='SAWBRIDGEWORTH', district='EPPING FOREST', county='ESSEX', ppd_category_type='A', record_status='A', pid='CM21 9PF, 100')


In [37]:
# For merging new data into the existing JSONLine file